In [5]:
!wget https://object-store.os-api.cci2.ecmwf.int/cci2-prod-cache-3/2025-09-01/fea05ca51bc5d8ee704a4aaddb69a95a.grib -O era5.grib

--2025-09-02 23:15:53--  https://object-store.os-api.cci2.ecmwf.int/cci2-prod-cache-3/2025-09-01/fea05ca51bc5d8ee704a4aaddb69a95a.grib
Resolving object-store.os-api.cci2.ecmwf.int (object-store.os-api.cci2.ecmwf.int)... 136.156.136.3
Connecting to object-store.os-api.cci2.ecmwf.int (object-store.os-api.cci2.ecmwf.int)|136.156.136.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36481497984 (34G) [application/x-grib]
Saving to: ‘era5.grib’

era5.grib           100%[===================>]  33.98G  10.8MB/s    in 77m 58s 

2025-09-03 00:33:52 (7.44 MB/s) - ‘era5.grib’ saved [36481497984/36481497984]



In [1]:
!pip install cfgrib xarray

In [9]:
import os

pressureLevel = 850
grib_path = os.path.join(os.getcwd(), 'era5.grib')
out_dir = f"/content/gphImages/{pressureLevel}"
os.makedirs(out_dir, exist_ok=True)

In [10]:
import io
import os
from datetime import datetime

import numpy as np
import xarray as xr
from PIL import Image


# Physical constants
STANDARD_GRAVITY_M_PER_S2: float = 9.80665


def resolve_paths(pressureLevel):
    """Return absolute paths for project root, data dir, grib path, and output dir."""
    here = os.path.dirname(os.path.abspath(__file__))
    root = os.path.abspath(os.path.join(here, os.pardir))
    data_dir = os.path.join(root, "data")
    grib_path = os.path.join(data_dir, "data.grib")
    out_dir = os.path.join(data_dir, "gphImages", pressureLevel)
    os.makedirs(out_dir, exist_ok=True)
    return root, data_dir, grib_path, out_dir


def open_era5_dataset(path: str) -> xr.Dataset:
    if not os.path.exists(path):
        raise FileNotFoundError(f"GRIB file not found: {path}")
    return xr.open_dataset(path, engine="cfgrib")


def select_gph_z(era5_ds: xr.Dataset, pressureLevel) -> xr.DataArray:
    if "z" not in era5_ds.variables:
        raise KeyError("Variable 'z' (geopotential) not found in dataset")
    z_da = era5_ds["z"]
    dims = set(z_da.dims)
    if "isobaricInhPa" in dims:
        da = z_da.sel(isobaricInhPa=pressureLevel)
    elif "level" in dims:
        da = z_da.sel(level=pressureLevel)
    else:
        da = z_da
    if "latitude" in da.coords and da.latitude.ndim == 1 and np.any(np.diff(da.latitude.values) < 0):
        da = da.sortby("latitude")
    return da


def to_minus180_180(lon_1d: np.ndarray, elev_m: np.ndarray):
    lon = lon_1d.copy()
    nx = lon.size
    dlon = float(np.round((lon[1] - lon[0]) * 1e6) / 1e6)
    if lon.min() >= -180 and lon.max() <= 180:
        return lon, elev_m
    shift = int(np.round((-180.0 - lon[0]) / dlon)) % nx
    elev_rot = np.roll(elev_m, shift=shift, axis=1)
    lon_rot = lon + shift * dlon
    lon_rot = ((lon_rot + 180.0) % 360.0) - 180.0
    order = np.argsort(lon_rot)
    lon_sorted = lon_rot[order]
    elev_sorted = elev_rot[:, order]
    return lon_sorted, elev_sorted


def encode_terrain_rgb_png(elev_m: np.ndarray, lat: np.ndarray, lon: np.ndarray):
    scaled = np.round((elev_m + 10000.0) / 0.1).astype(np.uint32)
    r = (scaled >> 16) & 255
    g = (scaled >> 8) & 255
    b = scaled & 255
    rgba = np.dstack([
        r.astype(np.uint8),
        g.astype(np.uint8),
        b.astype(np.uint8),
        np.full_like(r, 255, dtype=np.uint8),
    ])
    image = Image.fromarray(rgba, mode="RGBA")
    buf = io.BytesIO()
    image.save(buf, format="PNG")
    buf.seek(0)
    ny, nx = elev_m.shape
    min_lon = float(lon[0]) if lon[0] <= lon[-1] else float(lon[-1])
    max_lon = float(lon[-1]) if lon[-1] >= lon[0] else float(lon[0])
    min_lat = float(lat[0]) if lat[0] <= lat[-1] else float(lat[-1])
    max_lat = float(lat[-1]) if lat[-1] >= lat[0] else float(lat[0])
    bounds = (min_lon, min_lat, max_lon, max_lat)
    return buf.read(), bounds, nx, ny


def gphMain():
    # pressureLevel = 500
    # grib_path = "E:/datasets/era5-250,500,850,uv.grib"
    # out_dir = f"E:/datasets/gphImages/{pressureLevel}"
    # _, _, _, out_dir = resolve_paths(pressureLevel)

    ds = open_era5_dataset(grib_path)
    gphZ_data = select_gph_z(ds, pressureLevel)

    if "time" in gphZ_data.dims or "time" in gphZ_data.coords:
        time_coord = "time"
    elif "valid_time" in gphZ_data.coords:
        time_coord = "valid_time"
    else:
        raise KeyError("No time coordinate found (expected 'time' or 'valid_time')")

    lat = gphZ_data.latitude.values
    lon = gphZ_data.longitude.values

    times = gphZ_data[time_coord].values
    t0 = np.datetime_as_string(times[0], unit="h")
    tN = np.datetime_as_string(times[-1], unit="h")
    print(f"Dataset time range: {t0} .. {tN}")

    expected_start = datetime.strptime("2017080100", "%Y%m%d%H")
    expected_end = datetime.strptime("2017093023", "%Y%m%d%H")
    start_dt = np.datetime64(times[0]).astype("datetime64[h]")
    end_dt = np.datetime64(times[-1]).astype("datetime64[h]")
    covers_expected = (start_dt <= np.datetime64(expected_start)) and (end_dt >= np.datetime64(expected_end))
    print(f"Covers expected 2017080100..2017093023: {covers_expected}")

    total = len(times)
    for idx, t in enumerate(times, start=1):
        # Format YYYYMMDDHH
        ts = np.datetime_as_string(t, unit="h").replace("-", "").replace(":", "").replace("T", "")
        png_path = os.path.join(out_dir, f"gph_{ts}.png")
        if os.path.exists(png_path):
            if idx % 100 == 0:
                print(f"[{idx}/{total}] Exists, skipping: {os.path.basename(png_path)}")
            continue

        slice_da = (gphZ_data.sel({time_coord: np.datetime64(t)}) / STANDARD_GRAVITY_M_PER_S2)
        elev_m = slice_da.values.astype(np.float32)

        lon_fixed, elev_fixed = to_minus180_180(lon, elev_m)
        lat_work = lat.copy()
        if lat_work[0] < lat_work[-1]:
            elev_fixed = elev_fixed[::-1, :]
            lat_work = lat_work[::-1]

        png_bytes, _, _, _ = encode_terrain_rgb_png(elev_fixed, lat_work, lon_fixed)
        with open(png_path, "wb") as f:
            f.write(png_bytes)

        if idx % 50 == 0 or idx == total:
            print(f"[{idx}/{total}] Wrote {os.path.basename(png_path)}")


gphMain()




/usr/local/lib/python3.12/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/tmp/ipython-input-52172982.py:74: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  image = Image.fromarray(rgba, mode="RGBA")


Dataset time range: 2017-08-01T00 .. 2017-09-30T23
Covers expected 2017080100..2017093023: True
[50/1464] Wrote gph_2017080301.png
[100/1464] Wrote gph_2017080503.png
[150/1464] Wrote gph_2017080705.png
[200/1464] Wrote gph_2017080907.png
[250/1464] Wrote gph_2017081109.png
[300/1464] Wrote gph_2017081311.png
[350/1464] Wrote gph_2017081513.png
[400/1464] Wrote gph_2017081715.png
[450/1464] Wrote gph_2017081917.png
[500/1464] Wrote gph_2017082119.png
[550/1464] Wrote gph_2017082321.png
[600/1464] Wrote gph_2017082523.png
[650/1464] Wrote gph_2017082801.png
[700/1464] Wrote gph_2017083003.png
[750/1464] Wrote gph_2017090105.png
[800/1464] Wrote gph_2017090307.png
[850/1464] Wrote gph_2017090509.png
[900/1464] Wrote gph_2017090711.png
[950/1464] Wrote gph_2017090913.png
[1000/1464] Wrote gph_2017091115.png
[1050/1464] Wrote gph_2017091317.png
[1100/1464] Wrote gph_2017091519.png
[1150/1464] Wrote gph_2017091721.png
[1200/1464] Wrote gph_2017091923.png
[1250/1464] Wrote gph_2017092201.png

In [11]:
!zip -r /content/gphImages.zip /content/gphImages

  adding: content/gphImages/ (stored 0%)
  adding: content/gphImages/250/ (stored 0%)
  adding: content/gphImages/250/gph_2017091603.png (deflated 0%)
  adding: content/gphImages/250/gph_2017092406.png (deflated 0%)
  adding: content/gphImages/250/gph_2017081820.png (deflated 0%)
  adding: content/gphImages/250/gph_2017081900.png (deflated 0%)
  adding: content/gphImages/250/gph_2017091601.png (deflated 0%)
  adding: content/gphImages/250/gph_2017091403.png (deflated 0%)
  adding: content/gphImages/250/gph_2017092910.png (deflated 0%)
  adding: content/gphImages/250/gph_2017092316.png (deflated 0%)
  adding: content/gphImages/250/gph_2017090807.png (deflated 0%)
  adding: content/gphImages/250/gph_2017090806.png (deflated 0%)
  adding: content/gphImages/250/gph_2017092717.png (deflated 0%)
  adding: content/gphImages/250/gph_2017082610.png (deflated 0%)
  adding: content/gphImages/250/gph_2017083103.png (deflated 0%)
  adding: content/gphImages/250/gph_2017082917.png (deflated 0%)
  ad

In [17]:
import os

pressureLevel = 850
grib_path = os.path.join(os.getcwd(), 'era5.grib')
out_dir = f"/content/uv_images/{pressureLevel}"
os.makedirs(out_dir, exist_ok=True)

In [18]:
import io
import os
import argparse
from datetime import datetime

import numpy as np
import xarray as xr
from PIL import Image


def resolve_paths():
    """Return absolute paths for project root, data dir, uv grib path, and output dir."""
    here = os.path.dirname(os.path.abspath(__file__))
    root = os.path.abspath(os.path.join(here, os.pardir))
    data_dir = os.path.join(root, "data")
    grib_path = os.path.join(data_dir, "500hpa_uv_wind.grib")
    out_dir = os.path.join(data_dir, "uv_images")
    os.makedirs(out_dir, exist_ok=True)
    return root, data_dir, grib_path, out_dir


def open_dataset(path: str) -> xr.Dataset:
    if not os.path.exists(path):
        raise FileNotFoundError(f"UV GRIB file not found: {path}")
    # cfgrib engine handles GRIB; .grib path provided by user
    return xr.open_dataset(path, engine="cfgrib")


def select_level_component(ds: xr.Dataset, var_name: str, level_hpa: int) -> xr.DataArray:
    if var_name not in ds.variables:
        raise KeyError(f"Variable '{var_name}' not found in dataset")
    da = ds[var_name]
    dims = set(da.dims)
    if "isobaricInhPa" in dims:
        da = da.sel(isobaricInhPa=level_hpa)
    elif "level" in dims:
        da = da.sel(level=level_hpa)
    # Ensure latitude increases south->north
    if "latitude" in da.coords and da.latitude.ndim == 1 and np.any(np.diff(da.latitude.values) < 0):
        da = da.sortby("latitude")
    return da


def to_minus180_180(lon_1d: np.ndarray, field_2d: np.ndarray):
    """Convert lon from [0,360) to [-180,180) and roll columns accordingly."""
    lon = lon_1d.copy()
    nx = lon.size
    if nx < 2:
        return lon, field_2d
    dlon = float(np.round((lon[1] - lon[0]) * 1e6) / 1e6)
    if lon.min() >= -180 and lon.max() <= 180:
        return lon, field_2d
    shift = int(np.round((-180.0 - lon[0]) / dlon)) % nx
    rolled = np.roll(field_2d, shift=shift, axis=1)
    lon_rot = lon + shift * dlon
    lon_rot = ((lon_rot + 180.0) % 360.0) - 180.0
    order = np.argsort(lon_rot)
    lon_sorted = lon_rot[order]
    field_sorted = rolled[:, order]
    return lon_sorted, field_sorted


def scale_to_255(a: np.ndarray) -> np.ndarray:
    """Scale array to uint8 [0,255] using per-slice min-max. NaNs -> 0.

    If the slice is constant or empty, return mid-gray (127) where finite, else 0.
    """
    a = a.astype(np.float32)
    mask = np.isfinite(a)
    if not np.any(mask):
        return np.zeros_like(a, dtype=np.uint8)

    # TO DO: this is inconsistent scale across multiple different hours
    # if there is really high wind speed then every other wind will appear relatively slower
    # there should be a fixed scale here for accurate comparison
    vmin = float(np.min(a[mask]))
    vmax = float(np.max(a[mask]))
    if vmax <= vmin:
        out = np.full_like(a, 127, dtype=np.uint8)
        out[~mask] = 0
        return out
    scaled = (a - vmin) / (vmax - vmin)
    scaled = np.clip(scaled * 255.0, 0.0, 255.0)
    out = scaled.astype(np.uint8)
    out[~mask] = 0
    return out


def encode_uv_rg_png(u: np.ndarray, v: np.ndarray) -> bytes:
    """Encode two fields into PNG with U in red and V in green; B=0, A=255."""
    r = scale_to_255(u)
    g = scale_to_255(v)
    b = np.zeros_like(r, dtype=np.uint8)
    a = np.full_like(r, 255, dtype=np.uint8)
    rgba = np.dstack([r, g, b, a])
    image = Image.fromarray(rgba, mode="RGBA")
    buf = io.BytesIO()
    image.save(buf, format="PNG")
    buf.seek(0)
    return buf.read()


def uvMain():
    # pressureLevel = 500
    # grib_path = "."
    # out_dir = "../data/uv_images/{pressureLevel}"
    # _, _, grib_path, out_dir = resolve_paths()
    ds = open_dataset(grib_path)

    # Select 500 hPa components
    u_da = select_level_component(ds, "u", pressureLevel)
    v_da = select_level_component(ds, "v", pressureLevel)

    # Determine time coordinate
    if "time" in u_da.dims or "time" in u_da.coords:
        time_coord = "time"
    elif "valid_time" in u_da.coords:
        time_coord = "valid_time"
    else:
        raise KeyError("No time coordinate found (expected 'time' or 'valid_time')")

    # Coordinates
    lat = u_da.latitude.values
    lon = u_da.longitude.values

    # Normalize lon ordering for bounds/consistency and lat orientation (north->south rows)
    # We'll apply the same transforms to each time slice
    times = u_da[time_coord].values
    if times.size == 0:
        print("No time steps found in dataset")
        return

    # --- Happy path: fixed date window, hourly, inclusive ---
    start_np = np.datetime64("2017-08-01T00")
    end_np   = np.datetime64("2017-09-30T23")
    mask = (times >= start_np) & (times <= end_np)
    selected_times = times[mask]
    if selected_times.size == 0:
        t0 = np.datetime_as_string(times[0], unit="h")
        tN = np.datetime_as_string(times[-1], unit="h")
        print(f"No times within happy-path window (2017-08-01T00 .. 2017-09-30T23). "
              f"Dataset range is {t0} .. {tN}")
        return

    t0 = np.datetime_as_string(times[0], unit="h")
    tN = np.datetime_as_string(times[-1], unit="h")
    print(f"Dataset time range: {t0} .. {tN}")
    total = selected_times.size

    # Precompute lon/lat transform for bounds and orientation
    lon_fixed, _ = to_minus180_180(lon, np.zeros((lat.size, lon.size), dtype=np.float32))
    lat_work = lat.copy()
    flip_lat = False
    if lat_work[0] < lat_work[-1]:
        lat_work = lat_work[::-1]
        flip_lat = True

    for idx, t in enumerate(selected_times, start=1):
        ts = np.datetime_as_string(t, unit="h").replace("-", "").replace(":", "").replace("T", "")
        png_path = os.path.join(out_dir, f"uv_{ts}.png")

        # Extract slices
        u_sl = u_da.sel({time_coord: np.datetime64(t)})
        v_sl = v_da.sel({time_coord: np.datetime64(t)})
        u_vals = u_sl.values.astype(np.float32)
        v_vals = v_sl.values.astype(np.float32)

        # Ensure 2D and consistent orientation
        if u_vals.ndim != 2 or v_vals.ndim != 2:
            raise RuntimeError("Unexpected data shape for UV slice; expected 2D lat-lon")

        lon_u, u_fixed = to_minus180_180(lon, u_vals)
        lon_v, v_fixed = to_minus180_180(lon, v_vals)
        # Sanity: both lon transforms should be identical
        if lon_u.shape != lon_fixed.shape or not np.allclose(lon_u, lon_fixed):
            lon_fixed = lon_u
        if flip_lat:
            u_fixed = u_fixed[::-1, :]
            v_fixed = v_fixed[::-1, :]

        png_bytes = encode_uv_rg_png(u_fixed, v_fixed)

        with open(png_path, "wb") as f:
            f.write(png_bytes)

        if idx % 50 == 0 or idx == total:
            print(f"[{idx}/{total}] Wrote {os.path.basename(png_path)}")


uvMain()




/usr/local/lib/python3.12/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/tmp/ipython-input-1486617294.py:96: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  image = Image.fromarray(rgba, mode="RGBA")


Dataset time range: 2017-08-01T00 .. 2017-09-30T23
[50/1464] Wrote uv_2017080301.png
[100/1464] Wrote uv_2017080503.png
[150/1464] Wrote uv_2017080705.png
[200/1464] Wrote uv_2017080907.png
[250/1464] Wrote uv_2017081109.png
[300/1464] Wrote uv_2017081311.png
[350/1464] Wrote uv_2017081513.png
[400/1464] Wrote uv_2017081715.png
[450/1464] Wrote uv_2017081917.png
[500/1464] Wrote uv_2017082119.png
[550/1464] Wrote uv_2017082321.png
[600/1464] Wrote uv_2017082523.png
[650/1464] Wrote uv_2017082801.png
[700/1464] Wrote uv_2017083003.png
[750/1464] Wrote uv_2017090105.png
[800/1464] Wrote uv_2017090307.png
[850/1464] Wrote uv_2017090509.png
[900/1464] Wrote uv_2017090711.png
[950/1464] Wrote uv_2017090913.png
[1000/1464] Wrote uv_2017091115.png
[1050/1464] Wrote uv_2017091317.png
[1100/1464] Wrote uv_2017091519.png
[1150/1464] Wrote uv_2017091721.png
[1200/1464] Wrote uv_2017091923.png
[1250/1464] Wrote uv_2017092201.png
[1300/1464] Wrote uv_2017092403.png
[1350/1464] Wrote uv_2017092605.p

In [19]:
!zip -r /content/uv_images.zip /content/uv_images

  adding: content/uv_images/ (stored 0%)
  adding: content/uv_images/250/ (stored 0%)
  adding: content/uv_images/250/uv_2017082011.png (deflated 0%)
  adding: content/uv_images/250/uv_2017082112.png (deflated 0%)
  adding: content/uv_images/250/uv_2017092101.png (deflated 0%)
  adding: content/uv_images/250/uv_2017092401.png (deflated 0%)
  adding: content/uv_images/250/uv_2017080722.png (deflated 0%)
  adding: content/uv_images/250/uv_2017093009.png (deflated 0%)
  adding: content/uv_images/250/uv_2017080418.png (deflated 0%)
  adding: content/uv_images/250/uv_2017090801.png (deflated 0%)
  adding: content/uv_images/250/uv_2017091514.png (deflated 0%)
  adding: content/uv_images/250/uv_2017081009.png (deflated 0%)
  adding: content/uv_images/250/uv_2017091823.png (deflated 0%)
  adding: content/uv_images/250/uv_2017082704.png (deflated 0%)
  adding: content/uv_images/250/uv_2017091011.png (deflated 0%)
  adding: content/uv_images/250/uv_2017082418.png (deflated 0%)
  adding: content/

In [25]:
!zip -r /content/uv_images_850.zip /content/uv_images/850

  adding: content/uv_images/850/ (stored 0%)
  adding: content/uv_images/850/uv_2017082011.png (deflated 0%)
  adding: content/uv_images/850/uv_2017082112.png (deflated 0%)
  adding: content/uv_images/850/uv_2017092101.png (deflated 0%)
  adding: content/uv_images/850/uv_2017092401.png (deflated 0%)
  adding: content/uv_images/850/uv_2017080722.png (deflated 0%)
  adding: content/uv_images/850/uv_2017093009.png (deflated 0%)
  adding: content/uv_images/850/uv_2017080418.png (deflated 0%)
  adding: content/uv_images/850/uv_2017090801.png (deflated 0%)
  adding: content/uv_images/850/uv_2017091514.png (deflated 0%)
  adding: content/uv_images/850/uv_2017081009.png (deflated 0%)
  adding: content/uv_images/850/uv_2017091823.png (deflated 0%)
  adding: content/uv_images/850/uv_2017082704.png (deflated 0%)
  adding: content/uv_images/850/uv_2017091011.png (deflated 0%)
  adding: content/uv_images/850/uv_2017082418.png (deflated 0%)
  adding: content/uv_images/850/uv_2017080809.png (deflated

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
!cp /content/uv_images.zip /content/drive/MyDrive/


In [37]:
!du -h /content/gphImages.zip

3.3G	/content/gphImages.zip
